In [ ]:
#libraries 
import pandas as pd

from sklearn import  ensemble, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import RocCurveDisplay, r2_score,PredictionErrorDisplay, classification_report


import pickle
import csv

In [2]:
#data_download 
L= pd.read_csv("lip_forfinal.csv") #just the lipids 
Z= pd.DataFrame(pd.read_csv("CRvnoR_lip_forfinal.csv")) #clinical features
Z['response'] = Z['response'].replace('CR', 0).replace('NR', 1)

y = Z['response']
X = L

SEED = 42

##Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=SEED,stratify=y)

##apply feature pruning 
selected_features_names_CSV=open("20250210_pruned_features.csv", 'r')
file=csv.DictReader(selected_features_names_CSV)
pruned_features= []
for col in file:
    pruned_features.append(col['0'])


selected_features= X[pruned_features]
selected_features_names = selected_features.columns.tolist()

X_train_sel = X_train[selected_features_names]
X_test_sel = X_test[selected_features_names]

In [3]:
# load the lipid model from pickle file
model_pkl_file = "20250210_Lipid_GBC_.pkl"  
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)

# evaluate model 
y_predict = model.predict(X_test_sel)

# check results
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91        36
           1       0.67      0.91      0.77        11

    accuracy                           0.87        47
   macro avg       0.82      0.89      0.84        47
weighted avg       0.90      0.87      0.88        47



In [5]:
#Download the validation data 
Q2= pd.read_csv("Val_Col_lip_V2_adjusted___.csv") #just the lipids 
Z2= pd.DataFrame(pd.read_csv("Val_Col_lip_V2_adjusted.csv"))#clinical features

Z2['response'] = Z2['response'].replace('CR', 0).replace('NR', 1)
y2 = Z2['response']
X2 = Q2

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,  test_size=0.2, random_state=SEED,stratify=y2)

X2_sel = X2[selected_features_names]

X2_train_sel=X2_train[selected_features_names]
X2_test_sel=X2_test[selected_features_names]



In [6]:
classifier_output_for_survival= pd.DataFrame(model.predict_proba(X2[selected_features_names]))

# classifier_output_for_survival.to_csv('20250210_classification_output_for_survival_Val.csv')
# X_train.to_csv("Training_Val_20250210.csv")
# X_test.to_csv("Testing_Val_20250210.csv")
